# Predicting March Madness Matchup Scores
### Using past NCAA team data to predict how a school fares in the tournament

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import random
import math
import heapq
from scipy.stats import norm

## Creating Training Data

### Load in past tournament matchup data
**Initial Columns:** year, team1, score1, team2, score2, winner (team1/team2 with higher score)
**New Columns:** wins (number of times the winner won in the tournament), losses (number of times the winner lost in the tournament, either 0 or 1)

In [2]:
tournament = pd.read_csv('ncaa_matchups.csv')
# Create dataframe with tournamnet winners and number of wins (as well as losses)
wins = tournament.groupby(['year','winner']).count()[['team1']].rename(columns={'team1':'wins'})
wins['losses'] = np.where(wins['wins'] < 6, 1, 0)

# Create dataframe with tournament teams that didn't win a game
losses1 = tournament.loc[~tournament['team1'].isin(set(wins.index.get_level_values(1)))][['year',\
                                                        'team1']].rename(columns={'team1':'team'})
losses2 = tournament.loc[~tournament['team2'].isin(set(wins.index.get_level_values(1)))][['year',\
                                                        'team2']].rename(columns={'team2':'team'})

losses = losses1.append(losses2)
losses['losses_y'] = 1

wins.head()

wins  losses
year winner                   
2014 Arizona         3       1
     Baylor          2       1
     Connecticut     6       0
     Creighton       1       1
     Dayton          3       1

In [3]:
test = pd.merge(tournament, wins, how='left', on=['year','winner'])
test.head()

,year,team1,score1,team2,score2,winner,wins,losses
0,2019,Duke,85,North Dakota,62,Duke,3,1
1,2019,Virginia Commonwealth,58,Central Florida,73,Central Florida,1,1
2,2019,Mississippi State,76,Liberty,80,Liberty,1,1
3,2019,Virginia Tech,66,Saint Louis,52,Virginia Tech,2,1
4,2019,Maryland,79,Belmont,77,Maryland,1,1


### Create school-season dataset
Each row represents a season for an NCAA team. Columns indicate season averages (including opponent averages), overall record, and strength of schedule. **Source:** _Sports Reference - College Basketball_; Basic Team Stats, Advanced Team Stats, Basic Opponent Stats, Advanced Opponent Stats.

In [4]:
for yr in range(2014,2018):
    # Load data
    dfa = pd.read_csv('Data/Tm/tm'+str(yr)+'.csv')
    dfb = pd.read_csv('Data/Opp/opp'+str(yr)+'.csv')
    dfc = pd.read_csv('Data/TmA/tma'+str(yr)+'.csv')
    dfd = pd.read_csv('Data/OppA/oppa'+str(yr)+'.csv')
    df = pd.merge(pd.merge(pd.merge(dfa, dfb, how='inner', on=['school']), \
                  dfc, how='inner', on=['school']), dfd, how='inner', on=['school'])

    # Manipulate data
    cols = ['tm_pts','opp_pts','mp','fga','tpa','fta','orb','trb','ast','stl','blk','to','pf',\
          'opp_mp','opp_fga','opp_tpa','opp_fta','opp_orb','opp_trb','opp_ast','opp_stl','opp_blk','opp_to','opp_pf']
    # Normalize stats to reflect per-game averages
    df[cols] = df.apply(lambda row: row[cols]/row['gp'], axis=1)
    # Create column to store year
    df['year'] = yr
    # Subtract out tournament games when calculating each school's record
    df = pd.merge(df, wins, how='left', left_on=['year','school'], right_on=['year','winner'])
    cols = ['gp','w','aw']
    df[cols] = df.fillna(0).apply(lambda row: row[cols]-row['wins'], axis=1)
    cols = ['l','al']
    df[cols] = df.fillna(0).apply(lambda row: row[cols]-row['losses'], axis=1)
    df = pd.merge(df, losses, how='left', left_on=['year','school'], right_on=['year','team'])
    df[cols] = df.fillna(0).apply(lambda row: row[cols]-row['losses_y'], axis=1)
    # Calculate total and away records for each school
    df['rec'] = df.apply(lambda row: row['w']/(row['w']+row['l']), axis=1)
    df['arec'] = df.apply(lambda row: row['aw']/(row['aw']+row['al']), axis=1)
    # Drop unnecessary columns
    cols = ['gp','w','l','aw','al','wins','losses','losses_y','team']
    df = df.drop(cols, axis=1)

    # Store yearly dataframe
    df.to_csv('Data/All/df'+str(yr)+'.csv', index=False)

# Combine yearly dataframes into a single dataframe
df = pd.read_csv('Data/All/df2014.csv')
for yr in range(2015,2018):
    df_new = pd.read_csv('Data/All/df'+str(yr)+'.csv')
    df = df.append(df_new)
    
df.to_csv('df.csv')
df.head()

,school,srs,sos,tm_pts,opp_pts,mp,fga,fg_pct,tpa,tp_pct,...,opp_ast_pct,opp_stl_pct,opp_blk_pct,opp_efg_pct,opp_to_pct,opp_orb_pct,opp_ft_fga,year,rec,arec
0,Abilene Christian,-19.60,-4.12,71.419355,71.870968,40.483871,53.677419,0.443,19.322581,0.402,...,53.6,9.7,12.0,0.518,17.8,30.2,0.323,2014,0.354839,0.000000
1,Air Force,-4.08,1.71,66.000000,69.133333,40.333333,52.033333,0.435,21.766667,0.331,...,58.6,10.5,14.1,0.499,15.6,29.4,0.302,2014,0.400000,0.272727
2,Akron,1.16,-0.48,68.588235,66.941176,40.441176,54.558824,0.438,20.294118,0.346,...,46.7,9.9,9.1,0.479,16.7,32.5,0.259,2014,0.617647,0.428571
3,Alabama A&M,-13.86,-10.58,64.533333,67.000000,40.500000,53.366667,0.410,18.700000,0.324,...,45.3,9.8,8.9,0.452,17.3,34.3,0.365,2014,0.466667,0.312500
4,Alabama-Birmingham,0.78,-0.77,73.096774,70.322581,40.483871,61.903226,0.417,16.935484,0.301,...,50.1,9.9,9.1,0.465,13.5,29.8,0.201,2014,0.580645,0.500000


### Create tournament matchup dataset
Using the tournament matchup data and school data, create a new dataset in which one row includes one team's data (x) and its specific tournament opponent's data (y). There are 63 games played in a single tournament, so there are 63 rows per year, representing each matchup.

In [5]:
# Join school data (team1) on March Madness opponent data (team2)
ncaa_a = pd.merge(pd.merge(tournament, df, how='left', left_on=['year','team1'], right_on=['year','school']), \
                  df, how='inner', left_on=['year','team2'], right_on=['year','school'])
cols = ['team2','score2','school_x','school_y','winner']
ncaa_a = ncaa_a.drop(cols,axis=1).rename(columns={'team1':'team','score1':'score'})

# Join opponent data (team2) on March Madness school data (team1)
ncaa_b = pd.merge(pd.merge(tournament, df, how='left', left_on=['year','team2'], right_on=['year','school']), \
                  df, how='inner', left_on=['year','team1'], right_on=['year','school'])
cols = ['team1','score1','school_x','school_y','winner']
ncaa_b = ncaa_b.drop(cols,axis=1).rename(columns={'team2':'team','score2':'score'})

# Append both datasets
ncaa = ncaa_a.append(ncaa_b)
ncaa.to_csv('ncaa.csv')
ncaa.head()

,year,team,score,srs_x,sos_x,tm_pts_x,opp_pts_x,mp_x,fga_x,fg_pct_x,...,opp_trb_pct_y,opp_ast_pct_y,opp_stl_pct_y,opp_blk_pct_y,opp_efg_pct_y,opp_to_pct_y,opp_orb_pct_y,opp_ft_fga_y,rec_y,arec_y
0,2017,Villanova,76,23.80,9.28,77.194444,62.666667,40.000000,54.111111,0.495,...,55.5,43.9,9.3,7.9,0.499,18.5,34.6,0.215,0.571429,0.444444
1,2017,Wisconsin,84,19.40,9.40,72.378378,62.378378,40.540541,57.702703,0.455,...,51.8,49.9,7.7,11.8,0.510,14.3,30.2,0.196,0.666667,0.400000
2,2017,Virginia,76,20.63,10.93,66.058824,56.352941,40.588235,53.617647,0.460,...,48.4,46.2,6.8,7.4,0.526,17.8,27.6,0.283,0.852941,0.846154
3,2017,Florida,80,22.40,11.01,77.888889,66.500000,40.416667,58.833333,0.450,...,46.9,47.3,9.6,9.9,0.487,19.2,27.2,0.268,0.794118,0.666667
4,2017,Southern Methodist,65,18.71,4.45,74.257143,60.000000,40.000000,55.257143,0.473,...,49.7,53.6,7.9,8.5,0.501,15.3,31.2,0.172,0.735294,0.555556


## Use Genetic Algorithm to Select Features for Linear Model

### Establish objective functions
We will use AIC (a criterion that simultaneously rewards models with minimal error and penalizes overly complex models with too many features) to determine which features--and how many--to add to our model. AIC uses SSE (sum of squared errors), so we first define the function to calculate SSE.

In [6]:
def sum_sq_err(y_obs, y_pred):
    """
    inputs: y_obs, array of observed target values
            y_pred, array of predicted target values
    output: sse, sum of squared errors
    """
    return sum((y_obs-y_pred)**2)

def aic(y_obs, y_pred, k):
    """
    inputs: y_obs, array of observed target values
            y_pred, array of predicted target values
            k, number of features in model
    output: AIC (Akaike Information Criterion) for OLS, measure that rewards simple models
    """
    sse = sum_sq_err(y_obs, y_pred)
    n = len(y_pred)
    return 2*k + n*math.log(sse/n)

In [7]:
# Test above functions for SSE and AIC

S = len(ncaa.columns)
# Create random binary list that is 3 less than the length of the tournament matchup dataset
fs = np.random.randint(2, size=S-3)
# Convert binary list into list of indices (for which the binary output is 1)
features = []
for i in range(len(fs)):
    if fs[i]: features += [i]

# Use indices list to select features
# X contains the independent variables, y contains the dependent variable
X = ncaa.iloc[:,3:].iloc[:,features].values
y = ncaa['score'].values

# Fit a linear regression with X and y
lm = LinearRegression().fit(X, y)
lm.score(X,y)
lm.predict(X)

# Output the results of this modef selection(df_x, df_y, parents, threshold):
    """
    inputs: df_x, dataframe of dependent variable observations
            df_y, dataframe of independent variable observations
            parents, list of 'parent' vectors which determine features being used
            threshold, float on [0,1] which determines what portion of the population 'survives'
    output: portion of parents which are deemed most fit (based on AIC)
    """
    fitness = {}
    th = int(len(parents)*threshold)
    for s in parents:
        p = []
        for i in range(len(s)):
            if s[i]:
                p += [i]
        X = df_x.iloc[:,p].values
        y = df_y.values
        lm = LinearRegression().fit(X, y)
        fitness[aic(y, lm.predict(X), len(p))] = s
    best = heapq.nsmallest(th, fitness.keys())
    return [fitness[x] for x in best]

def crossover(parents, O):
    """
    inputs: parents, list of 'parent' vectors which determine features being used
            O, number of offspring generated
    output: offspring, list of 'child' vectors generated from parents
    """
    if len(parents) < 2:
        return
    l = len(parents[0])
    offspring = []
    for i in range(O):
        inds = np.random.choice(len(parents), 2, replace=False)
        cutoff = np.random.randint(1,l-1)
        offspring += [np.append(parents[inds[0]][:cutoff],parents[inds[1]][cutoff:])]
    return offspring

def mutation(offspring, r, P):
    """
    inputs: offspring, new generation of vectors which determine features being used
            r, float on [0,1] - rate at which mutation occurs
            P, number of parameters to choose from
    output: new list of features which has some random removals and additions
    """
    if len(offspring) < 1 or r > 1 or r < 0:
        return
    mutated = []
    for c in offspring:
        m = np.copy(c)
        for i in range(P):
            # Mutate with rate, r
            if np.random.choice([True,False], 1, p=[r,1-r]):
                m[i] = 1-c[i]
        mutated += [m]
    return mutateddel
print('Lenfth of dataframe:', S)
print('Number of features:', sum(fs))
print('SSE:', sum_sq_err(y, lm.predict(X)))
print('AIC:', aic(y, lm.predict(X), len(features)))

Lenfth of dataframe: 123
Number of features: 58
SSE: 39048.942522685466
AIC: 2308.397382599436


### Write definitions to perform selection, crossover, and mutation steps
**Selection:** Choose the best k percentage of candidate models based on AIC. 
**Crossover:** With pairs of randomly selected best candidates (parents), reproduce new candidates (children) by combining features of both parents. 
**Mutation:** With all the newly produced children candidates, randomly change some of the features.

In [8]:
def selection(df_x, df_y, parents, threshold):
    """
    inputs: df_x, dataframe of dependent variable observations
            df_y, dataframe of independent variable observations
            parents, list of 'parent' vectors which determine features being used
            threshold, float on [0,1] which determines what portion of the population 'survives'
    output: portion of parents which are deemed most fit (based on AIC)
    """
    fitness = {}
    th = int(len(parents)*threshold)
    for s in parents:
        p = []
        for i in range(len(s)):
            if s[i]:
                p += [i]
        X = df_x.iloc[:,p].values
        y = df_y.values
        lm = LinearRegression().fit(X, y)
        fitness[aic(y, lm.predict(X), len(p))] = s
    best = heapq.nsmallest(th, fitness.keys())
    return [fitness[x] for x in best]

def crossover(parents, O):
    """
    inputs: parents, list of 'parent' vectors which determine features being used
            O, number of offspring generated
    output: offspring, list of 'child' vectors generated from parents
    """
    if len(parents) < 2:
        return
    l = len(parents[0])
    offspring = []
    for i in range(O):
        inds = np.random.choice(len(parents), 2, replace=False)
        cutoff = np.random.randint(1,l-1)
        offspring += [np.append(parents[inds[0]][:cutoff],parents[inds[1]][cutoff:])]
    return offspring

def mutation(offspring, r, P):
    """
    inputs: offspring, new generation of vectors which determine features being used
            r, float on [0,1] - rate at which mutation occurs
            P, number of parameters to choose from
    output: new list of features which has some random removals and additions
    """
    if len(offspring) < 1 or r > 1 or r < 0:
        return
    mutated = []
    for c in offspring:
        m = np.copy(c)
        for i in range(P):
            # Mutate with rate, r
            if np.random.choice([True,False], 1, p=[r,1-r]):
                m[i] = 1-c[i]
        mutated += [m]
    return mutated

In [9]:
# Test the above functions to perform one step of the evolution algorithm

DFx = ncaa.iloc[:,3:]
DFy = ncaa['score']

N = 10
P = len(DFx.columns)
ps = np.random.randint(2, size=(N,P))
t = 0.2

print('Select parents:', selection(DFx, DFy, ps, t))

group = selection(DFx, DFy, ps, t)
print('Crossover parent genes to create child:', crossover(group, int(N-N*t))[0])


children = crossover(group, int(N-N*t))
print('Mutate child genes:', mutation([children[0]], 1/P, 94)[0])

Select parents: [array([1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1]), array([1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0])]
Crossover parent genes to create child: [1 1 0 0 1 0 0 1 0 1 1 0 0 0 1 1 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 1 1 1 1 0 0
 1 1 0 1 1 1 0 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1 1 0 1 0 1 1 1 1

### Create the evolutionary process (genetic algorithm)
Use the above definitions (selection, crossover, mutation) to create an evolutionary process that chooses the best features for the model in G generations. Test on 10 generations with games played as the dependent variable.

In [10]:
def evolution(df_x, df_y, G):
    """
    inputs: df_x, dataframe of dependent variable observations
            df_y, dataframe of independent variable observations
            G, number of generations
    output: most fit feature vector after G generations
    """
    N = 500
    t = 0.2
    P = len(df_x.columns)
    r = 1/P
    
    # Initialize G0 of parents (randomly select features for N parents)
    ps = np.random.randint(2, size=(N,P))
    
    # Go through evolutionary process (selection, crossover, mutation) for G generations
    gen = ps
    num_children = int(N-N*t)
    for g in range(G):
        survivors = selection(df_x, df_y, gen, t)
        children = mutation(crossover(survivors, num_children), r, P)
        gen = survivors + children
        
    best = selection(df_x, df_y, gen, 1/N)[0]
    return best

In [20]:
# Test genetic algorithm (evolution) and copy features to a file

DFx = ncaa.iloc[:,3:]
DFy = ncaa['score']

fs = DFx.columns
use = evolution(DFx, DFy, 10)

features = []
for i in range(len(fs)):
    if use[i]:
        features += [fs[i]]
print('Selected Features:', features)

with open('featuresG10.txt', 'w') as f:
    for item in features:
        f.write("%s\n" % item)

['sos_x', 'mp_x', 'fta_x', 'opp_tpa_x', 'opp_tp_pct_x', 'opp_fta_x', 'opp_orb_x', 'opp_ast_x', 'opp_to_x', 'pace_x', 'ortg_x', 'ftr_x', 'ts_pct_x', 'ast_pct_x', 'opp_ftr_x', 'opp_tpar_x', 'opp_ts_pct_x', 'opp_to_pct_x', 'rec_x', 'arec_x', 'sos_y', 'tm_pts_y', 'opp_pts_y', 'tpa_y', 'fta_y', 'orb_y', 'ast_y', 'opp_tpa_y', 'opp_tp_pct_y', 'opp_fta_y', 'ortg_y', 'ast_pct_y', 'orb_pct_y', 'opp_ortg_y', 'opp_ts_pct_y', 'opp_trb_pct_y', 'opp_efg_pct_y', 'opp_to_pct_y', 'opp_orb_pct_y', 'opp_ft_fga_y']


### Select features to train model for different dependent variables (multiple stat categories)
**WARNING:** Runs for a very long time (hours) with large number of generations (recommended at least 1000 for acccurate results).

In [132]:
# Set number of generations for genetic algorithm
num_gen = 10000

# Run genetic algorithm for each dependent variable of interest
DFx = ncaa.iloc[:,3:]
DFy = ncaa['score']

fs = DFx.columns
# Run evolutionary algorithm for desired nnumber of generations
use = evolution(DFx, DFy, num_gen)

features = []
for i in range(len(fs)):
    if use[i]:
        features += [fs[i]]
print(features)

with open('Features/featuresG'+str(num_gen)+'.txt', 'w') as f:
    for item in features:
        f.write("%s\n" % item)

['sos_x', 'fg_pct_x', 'ast_x', 'opp_mp_x', 'opp_fga_x', 'opp_fta_x', 'ts_pct_x', 'ast_pct_x', 'opp_ftr_x', 'opp_tpar_x', 'rec_x', 'arec_x', 'sos_y', 'tm_pts_y', 'opp_pts_y', 'fta_y', 'opp_fga_y', 'opp_orb_y', 'ortg_y', 'opp_ortg_y', 'opp_ftr_y', 'opp_ts_pct_y', 'opp_to_pct_y']


In [11]:
features = pd.read_csv('Features/featuresG10000.txt', header=None)

X = ncaa.iloc[:,3:][features[0]].values
y = ncaa['score'].values

lm = LinearRegression().fit(X, y)

ncaa_train = pd.merge(pd.merge(tournament, df, how='left', left_on=['year','team1'], right_on=['year','school']), \
                  df, how='inner', left_on=['year','team2'], right_on=['year','school'])
cols = ['school_x','school_y']
ncaa_train = ncaa_train.drop(cols,axis=1)

fs1 = features[0].values
fs2 = [x[:-1]+'y' if x[-1] == 'x' else x[:-1]+'x' for x in fs1]

ncaa_train['pscore1'] = lm.intercept_
for i in range(len(features[0].values)):
    ncaa_train['pscore1'] = ncaa_train.apply(lambda row: row['pscore1'] + row[fs1[i]]*lm.coef_[i], axis=1)
ncaa_train['pscore2'] = lm.intercept_
for i in range(len(features[0].values)):
    ncaa_train['pscore2'] = ncaa_train.apply(lambda row: row['pscore2'] + row[fs2[i]]*lm.coef_[i], axis=1)
    
ncaa_train['pwinner'] = ncaa_train.apply(lambda row: row['team1'] if row['pscore1'] > row['pscore2'] else row['team2'], axis=1)
ncaa_train['correct'] = ncaa_train.apply(lambda row: 1 if row['winner'] == row['pwinner'] else 0, axis=1)

sse = sum_sq_err(y, lm.predict(X))

s = math.sqrt(sse/len(y))

ncaa_train['m'] = ncaa_train['pscore2']-ncaa_train['pscore1']
ncaa_train['sd'] = math.sqrt(2*s**2)

ncaa_train['probability'] = norm.cdf(0, ncaa_train['m'], ncaa_train['sd'])

cols = ['year','team1','team2','score1','score2','pscore1','pscore2','winner','pwinner','correct','probability']

ncaa_train[cols].to_csv('training_results.csv', index=False)
ncaa_train[cols].head()

,year,team1,team2,score1,score2,pscore1,pscore2,winner,pwinner,correct,probability
0,2017,Villanova,Mount St. Mary's,76,56,86.305109,58.504732,Villanova,Villanova,1,0.989052
1,2017,Wisconsin,Virginia Tech,84,74,81.350665,67.399095,Wisconsin,Wisconsin,1,0.874995
2,2017,Virginia,North Carolina-Wilmington,76,71,78.612226,70.755004,Virginia,Virginia,1,0.741456
3,2017,Florida,East Tennessee State,80,65,83.619691,65.507415,Florida,Florida,1,0.932332
4,2017,Southern Methodist,Southern California,65,66,72.232333,64.118577,Southern California,Southern Methodist,0,0.748249


## Test Linear Model
Use 2018 and 2019 tournament data to test the performance of this linear model.

### Create school-season dataset with 2018 and 2019 data
Each row represents a season for an NCAA team. Columns indicate season averages (including opponent averages), overall record, and strength of schedule.

In [12]:
for yr in range(2018,2020):
    # Load data
    dfa = pd.read_csv('Data/Tm/tm'+str(yr)+'.csv')
    dfb = pd.read_csv('Data/Opp/opp'+str(yr)+'.csv')
    dfc = pd.read_csv('Data/TmA/tma'+str(yr)+'.csv')
    dfd = pd.read_csv('Data/OppA/oppa'+str(yr)+'.csv')
    df = pd.merge(pd.merge(pd.merge(dfa, dfb, how='inner', on=['school']), \
                  dfc, how='inner', on=['school']), dfd, how='inner', on=['school'])

    # Manipulate data
    cols = ['tm_pts','opp_pts','mp','fga','tpa','fta','orb','trb','ast','stl','blk','to','pf',\
          'opp_mp','opp_fga','opp_tpa','opp_fta','opp_orb','opp_trb','opp_ast','opp_stl','opp_blk','opp_to','opp_pf']
    # Normalize stats to reflect per-game averages
    df[cols] = df.apply(lambda row: row[cols]/row['gp'], axis=1)
    # Create column to store year
    df['year'] = yr
    # Subtract out tournament games when calculating each school's record
    df = pd.merge(df, wins, how='left', left_on=['year','school'], right_on=['year','winner'])
    cols = ['gp','w','aw']
    df[cols] = df.fillna(0).apply(lambda row: row[cols]-row['wins'], axis=1)
    cols = ['l','al']
    df[cols] = df.fillna(0).apply(lambda row: row[cols]-row['losses'], axis=1)
    df = pd.merge(df, losses, how='left', left_on=['year','school'], right_on=['year','team'])
    df[cols] = df.fillna(0).apply(lambda row: row[cols]-row['losses_y'], axis=1)
    # Calculate total and away records for each school
    df['rec'] = df.apply(lambda row: row['w']/(row['w']+row['l']), axis=1)
    df['arec'] = df.apply(lambda row: row['aw']/(row['aw']+row['al']), axis=1)
    # Drop unnecessary columns
    cols = ['gp','w','l','aw','al','wins','losses','losses_y','team']
    df = df.drop(cols, axis=1)

    # Store yearly dataframe
    df.to_csv('Data/All/df'+str(yr)+'.csv', index=False)

# Combine yearly dataframes into a single dataframe
df = pd.read_csv('Data/All/df2018.csv')
df_new = pd.read_csv('Data/All/df2019.csv')
df = df.append(df_new)

df.head()

,school,srs,sos,tm_pts,opp_pts,mp,fga,fg_pct,tpa,tp_pct,...,opp_ast_pct,opp_stl_pct,opp_blk_pct,opp_efg_pct,opp_to_pct,opp_orb_pct,opp_ft_fga,year,rec,arec
0,Abilene Christian,-9.14,-6.82,73.718750,71.218750,40.31250,58.562500,0.464,20.500000,0.325,...,48.1,8.9,7.5,0.499,19.5,28.4,0.298,2018,0.500000,0.400000
1,Air Force,-4.31,1.72,68.516129,72.387097,40.16129,56.483871,0.419,24.354839,0.331,...,57.2,8.3,10.6,0.549,18.3,27.8,0.234,2018,0.387097,0.230769
2,Akron,-6.82,-1.92,71.750000,75.343750,40.46875,57.000000,0.435,26.593750,0.358,...,51.1,9.0,11.7,0.532,16.2,27.9,0.277,2018,0.437500,0.166667
3,Alabama A&M,-23.97,-8.04,60.419355,76.354839,40.16129,54.741935,0.397,19.387097,0.303,...,52.2,11.1,13.8,0.540,14.4,32.1,0.204,2018,0.096774,0.052632
4,Alabama-Birmingham,4.90,-0.65,76.848485,69.787879,40.30303,58.696970,0.488,19.575758,0.345,...,53.2,10.0,7.7,0.498,15.9,24.6,0.183,2018,0.606061,0.454545


### Create tournament matchup dataset and predict with pre-trained model
Using the trained model, predict the results on 2018 and 2019 tournament matchup data. **RESULTS:** 94/124 matchup winners predicted correctly (75.8%). The standord error (SE) of the predicted score in a given game is 12.13 points.

In [13]:
features = pd.read_csv('Features/featuresG10000.txt', header=None)

# Use training set to find the best linear model (same model as in previous steps)
X = ncaa.iloc[:,3:][features[0]].values
y = ncaa['score'].values

lm = LinearRegression().fit(X, y)

# Create test set with 2018 and 2019 data
ncaa_test = pd.merge(pd.merge(tournament, df, how='left', left_on=['year','team1'], right_on=['year','school']), \
                  df, how='inner', left_on=['year','team2'], right_on=['year','school'])
cols = ['school_x','school_y']
ncaa_test = ncaa_test.drop(cols,axis=1)

# Get inverse features to calculate opponent's score
fs1 = features[0].values
fs2 = [x[:-1]+'y' if x[-1] == 'x' else x[:-1]+'x' for x in fs1]

# Predict on test dataset with the trained model
ncaa_test['pscore1'] = lm.intercept_
for i in range(len(features[0].values)):
    ncaa_test['pscore1'] = ncaa_test.apply(lambda row: row['pscore1'] + row[fs1[i]]*lm.coef_[i], axis=1)
ncaa_test['pscore2'] = lm.intercept_
for i in range(len(features[0].values)):
    ncaa_test['pscore2'] = ncaa_test.apply(lambda row: row['pscore2'] + row[fs2[i]]*lm.coef_[i], axis=1)
    
ncaa_test['pwinner'] = ncaa_test.apply(lambda row: row['team1'] if row['pscore1'] > row['pscore2'] else row['team2'], axis=1)
ncaa_test['correct'] = ncaa_test.apply(lambda row: 1 if row['winner'] == row['pwinner'] else 0, axis=1)

# Calculate SSE for the purposes of calculating SE (will be used as SD for predictions)
sse = sum_sq_err(y, lm.predict(X))

# Calculate and print the SE
s = math.sqrt(sse/len(y))
print('SE:', math.sqrt(2*s**2))

# Store the predicted difference of scores as the mean (EV) and the standard error as the standard deviation (SD)
ncaa_test['m'] = ncaa_test['pscore2']-ncaa_test['pscore1']
ncaa_test['sd'] = math.sqrt(2*s**2)

# Calculate the probability of team1 winning, using the above mean and standard deviation
ncaa_test['probability'] = norm.cdf(0, ncaa_test['m'], ncaa_test['sd'])

cols = ['year','team1','team2','score1','score2','pscore1','pscore2','winner','pwinner','correct','probability']

# Write results of predictions to a new file
ncaa_test[cols].to_csv('test_results.csv', index=False)
ncaa_test[cols].head()

SE: 12.128350684739


,year,team1,team2,score1,score2,pscore1,pscore2,winner,pwinner,correct,probability
0,2019,Duke,North Dakota,85,62,92.126282,59.153843,Duke,Duke,1,0.996722
1,2019,Virginia Commonwealth,Central Florida,58,73,64.404200,62.189607,Central Florida,Virginia Commonwealth,0,0.572443
2,2019,Duke,Central Florida,77,76,70.919180,59.439388,Duke,Duke,1,0.828060
3,2019,Mississippi State,Liberty,76,80,75.988676,73.364722,Liberty,Mississippi State,0,0.585642
4,2019,Virginia Tech,Saint Louis,66,52,70.039867,58.013337,Virginia Tech,Virginia Tech,1,0.839305


## Predict 2020 Tournament Results

### Show the likeliest matchup winner of every team in the tournament
Predict winners in the first round of the 2020 tournament. Use those predicted winners (and the pre-determined bracket structure) to predict the next matchup, and then predict the winners of this hypothetical matchup. Repeat until there is only one team left in the tournament. Return the number of predicted games won for each team in the tournament. This is called the "likeliest outcome" because, according to the model, this is the likeliest bracket to be 100% correct.

In [14]:
# Load data
dfa = pd.read_csv('Data/Tm/tm2020.csv')
dfb = pd.read_csv('Data/Opp/opp2020.csv')
dfc = pd.read_csv('Data/TmA/tma2020.csv')
dfd = pd.read_csv('Data/OppA/oppa2020.csv')
df = pd.merge(pd.merge(pd.merge(dfa, dfb, how='inner', on=['school']), \
              dfc, how='inner', on=['school']), dfd, how='inner', on=['school'])

# Manipulate data
cols = ['tm_pts','opp_pts','mp','fga','tpa','fta','orb','trb','ast','stl','blk','to','pf',\
      'opp_mp','opp_fga','opp_tpa','opp_fta','opp_orb','opp_trb','opp_ast','opp_stl','opp_blk','opp_to','opp_pf']
# Normalize stats to reflect per-game averages
df[cols] = df.apply(lambda row: row[cols]/row['gp'], axis=1)
# Create column to store year
df['year'] = yr
# Subtract out tournament games when calculating each school's record
df = pd.merge(df, wins, how='left', left_on=['year','school'], right_on=['year','winner'])
cols = ['gp','w','aw']
df[cols] = df.fillna(0).apply(lambda row: row[cols]-row['wins'], axis=1)
cols = ['l','al']
df[cols] = df.fillna(0).apply(lambda row: row[cols]-row['losses'], axis=1)
df = pd.merge(df, losses, how='left', left_on=['year','school'], right_on=['year','team'])
df[cols] = df.fillna(0).apply(lambda row: row[cols]-row['losses_y'], axis=1)
# Calculate total and away records for each school
df['rec'] = df.apply(lambda row: row['w']/(row['w']+row['l']), axis=1)
df['arec'] = df.apply(lambda row: row['aw']/(row['aw']+row['al']), axis=1)
# Drop unnecessary columns
cols = ['gp','w','l','aw','al','wins','losses','losses_y','team']
df = df.drop(cols, axis=1)

# Store yearly dataframe
df.to_csv('Data/All/df2020.csv', index=False)

In [201]:
df19 = pd.read_csv('Data/All/df2019.csv')
bracket19 = pd.read_csv('bracket2019.csv')
features = pd.read_csv('Features/featuresG10000.txt', header=None)

# Use training set to find the best linear model (same model as in previous steps)
X = ncaa.iloc[:,3:][features[0]].values
y = ncaa['score'].values

lm = LinearRegression().fit(X, y)

# Get inverse features to calculate opponent's score
fs1 = [f[:-2] for f in features[0].values if f[-1] == 'x']
fs2 = [f[:-2] for f in features[0].values if f[-1] == 'y']

d = {}
for team in bracket19['team'].values:
    d[team] = 0

i = 2
while len(bracket19) > 1:
    teams = bracket19.nsmallest(i,'tournament_num').nlargest(2,'tournament_num').values
    team1 = teams[0][0]
    team2 = teams[1][0]
    pscore1 = lm.intercept_
    j = 0
    for f in fs1:
        pscore1 = pscore1 + df19[(df19['school']==team1)][f].values[0]*lm.coef_[j]
        j += 1
    for f in fs2:
        pscore1 = pscore1 + df19[(df19['school']==team2)][f].values[0]*lm.coef_[j]
        j += 1
    pscore2 = lm.intercept_
    j = 0
    for f in fs1:
        pscore2 = pscore2 + df19[(df19['school']==team2)][f].values[0]*lm.coef_[j]
        j += 1
    for f in fs2:
        pscore2 = pscore2 + df19[(df19['school']==team1)][f].values[0]*lm.coef_[j]
        j += 1
    if pscore1 > pscore2:
        bracket19 = bracket19.drop([teams[1][1]])
        d[team1] += 1
    else:
        bracket19 = bracket19.drop([teams[0][1]])
        d[team2] += 1
    if i >= len(bracket19): 
        i = 1
    i += 1
        
tournament19 = pd.DataFrame.from_dict(d, orient='index').rename(columns={0:'wins'})
tournament19.to_csv('likeliest_outcome_19.csv')
tournament19.head()

,wins
Duke,4
North Dakota State,0
Virginia Commonwealth,1
Central Florida,0
Mississippi State,1


### Simuate the probability of each team making each round of the tournament
Predict the probability of each team winning its first matchup. Generate a random number between 0 and 1, and if that random number is less than the probability, the team wins. Use those predicted winners (and the pre-determined bracket structure) to predict the next matchup, and then predict the winners of this hypothetical matchup with the same method. Do this until there is one team left in the tournament, and then repeat the whole process N times, logging how many times each team makes it to each round. Use this to produce the "probabilistic results," the chance of each team making it to each round. **WARNING:** Runs for a very long time when N is large.

In [199]:
df19 = pd.read_csv('Data/All/df2019.csv')
bracket19 = pd.read_csv('bracket2019.csv')
features = pd.read_csv('Features/featuresG10000.txt', header=None)

N = 10000
se = 12.129582155889153

# Use training set to find the best linear model (same model as in previous steps)
X = ncaa.iloc[:,3:][features[0]].values
y = ncaa['score'].values

lm = LinearRegression().fit(X, y)

# Get inverse features to calculate opponent's score
fs1 = [f[:-2] for f in features[0].values if f[-1] == 'x']
fs2 = [f[:-2] for f in features[0].values if f[-1] == 'y']

d = {}
for team in bracket19['team'].values:
    d[team] = [0,0,0,0,0,0]

for n in range(N):
    bracket19 = pd.read_csv('bracket2019.csv')
    i = 2
    r = 0
    while len(bracket19) > 1:
        teams = bracket19.nsmallest(i,'tournament_num').nlargest(2,'tournament_num').values
        team1 = teams[0][0]
        team2 = teams[1][0]
        pscore1 = lm.intercept_
        j = 0
        for f in fs1:
            pscore1 = pscore1 + df19[(df19['school']==team1)][f].values[0]*lm.coef_[j]
            j += 1
        for f in fs2:
            pscore1 = pscore1 + df19[(df19['school']==team2)][f].values[0]*lm.coef_[j]
            j += 1
        pscore2 = lm.intercept_
        j = 0
        for f in fs1:
            pscore2 = pscore2 + df19[(df19['school']==team2)][f].values[0]*lm.coef_[j]
            j += 1
        for f in fs2:
            pscore2 = pscore2 + df19[(df19['school']==team1)][f].values[0]*lm.coef_[j]
            j += 1
        pwin = norm.cdf(0, pscore2-pscore1, se)   
        if random.random() < pwin:
            bracket19 = bracket19.drop([teams[1][1]])
            d[team1][r] += 1
        else:
            bracket19 = bracket19.drop([teams[0][1]])
            d[team2][r] += 1
        if i >= len(bracket19): 
            i = 1
            r += 1
        i += 1
        
tournament19 = pd.DataFrame.from_dict(d, orient='index').div(N).rename(columns=\
                            {0:'R32',1:'Sweet 16',2:'Elite 8',3:'Final 4',4:'Final',5:'Champion'})
tournament19.to_csv('probablistic_results_19.csv')
tournament19.head()

,0,1,2,3,4,5
Duke,0.9797,0.7823,0.5878,0.3655,0.1462,0.0908
North Dakota State,0.0203,0.0039,0.0005,0.0000,0.0000,0.0000
Virginia Commonwealth,0.5729,0.1339,0.0625,0.0221,0.0045,0.0018
Central Florida,0.4271,0.0799,0.0347,0.0102,0.0016,0.0003
Mississippi State,0.5777,0.2744,0.0823,0.0278,0.0048,0.0017


In [15]:
# matchups19 = tournament[tournament['year']==2019][['team1','team2','score1','score2']]
matchups20 = pd.read_csv('conf_matchups.csv')
features = pd.read_csv('Features/featuresG10000.txt', header=None)

df20 = pd.read_csv('Data/All/df2020.csv')

# Use training set to find the best linear model (same model as in previous steps)
X = ncaa.iloc[:,3:][features[0]].values
y = ncaa['score'].values

lm = LinearRegression().fit(X, y)

# Create prediction set with 2020 data
ncaa_pred = pd.merge(pd.merge(matchups20, df20, how='left', left_on=['team1'], right_on=['school']), \
                  df20, how='inner', left_on=['team2'], right_on=['school'])
cols = ['school_x','school_y']
ncaa_pred = ncaa_pred.drop(cols,axis=1)

# Get inverse features to calculate opponent's score
fs1 = features[0].values
fs2 = [x[:-1]+'y' if x[-1] == 'x' else x[:-1]+'x' for x in fs1]

# Predict on test dataset with the trained model
ncaa_pred['pscore1'] = lm.intercept_
for i in range(len(features[0].values)):
    ncaa_pred['pscore1'] = ncaa_pred.apply(lambda row: row['pscore1'] + row[fs1[i]]*lm.coef_[i], axis=1)
ncaa_pred['pscore2'] = lm.intercept_
for i in range(len(features[0].values)):
    ncaa_pred['pscore2'] = ncaa_pred.apply(lambda row: row['pscore2'] + row[fs2[i]]*lm.coef_[i], axis=1)
    
# Calculate SSE for the purposes of calculating SE (will be used as SD for predictions)
sse = sum_sq_err(y, lm.predict(X))

# Calculate the SE
s = math.sqrt(sse/len(y))

# Store the predicted difference of scores as the mean (EV) and the standard error as the standard deviation (SD)
ncaa_pred['m'] = ncaa_pred['pscore2']-ncaa_pred['pscore1']
ncaa_pred['se'] = s
ncaa_pred['sd'] = math.sqrt(2*s**2)

# Predict the winner of the match based on the scores
ncaa_pred['pwinner'] = ncaa_pred.apply(lambda row: row['team1'] if row['pscore1'] > row['pscore2'] else row['team2'], axis=1)

# Calculate the probability of team1 winning, using the above mean and standard deviation
ncaa_pred['probability'] = norm.cdf(0, ncaa_pred['m'], ncaa_pred['sd'])

# Predict the O/U
ncaa_pred['o/u_25'] = ncaa_pred.apply(lambda row: row['pscore1'] + row['pscore2'] - 0.67*row['sd'], axis=1)
ncaa_pred['o/u_75'] = ncaa_pred.apply(lambda row: row['pscore1'] + row['pscore2'] + 0.67*row['sd'], axis=1)

# Predict the team1 score differential
ncaa_pred['diff_25'] = ncaa_pred.apply(lambda row: row['pscore2'] - row['pscore1'] - 0.67*row['sd'], axis=1)
ncaa_pred['diff_75'] = ncaa_pred.apply(lambda row: row['pscore2'] - row['pscore1'] + 0.67*row['sd'], axis=1)

cols = ['team1','team2','pscore1','pscore2','se','pwinner','probability','o/u_25','o/u_75','diff_25','diff_75']

# Write results of predictions to a new file
ncaa_pred[cols].to_csv('game_predictions_20.csv', index=False)
ncaa_pred[cols].head()

,team1,team2,pscore1,pscore2,se,pwinner,probability,o/u_25,o/u_75,diff_25,diff_75
0,Wake Forest,Pittsburgh,69.937010,72.531380,8.576039,Pittsburgh,0.415309,134.342395,150.594385,-5.531626,10.720364
1,North Carolina State,Pittsburgh,73.166806,67.522979,8.576039,North Carolina State,0.679157,132.563791,148.815781,-13.769822,2.482168
2,Virginia Tech,North Carolina,75.822891,67.200806,8.576039,Virginia Tech,0.761428,134.897702,151.149692,-16.748080,-0.496090
3,Clemson,Miami (FL),73.370136,69.455554,8.576039,Clemson,0.626563,134.699695,150.951685,-12.040576,4.211414
4,Notre Dame,Boston College,75.517722,62.789278,8.576039,Notre Dame,0.853021,130.181005,146.432995,-20.854440,-4.602450
